In [117]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn
from scipy.stats import poisson,skellam

world_cup_matches = pd.read_csv("C:\\Users\\ianfr\\Documents\\dev\\inteligencia-computacional\\dataset\\results.csv")
world_cup_matches['year'] = world_cup_matches['date'].str[:4].astype(int)
world_cup_matches['month'] = world_cup_matches['date'].str[5:7].astype(int)
world_cup_matches['day'] = world_cup_matches['date'].str[8:].astype(int)
world_cup_matches = world_cup_matches[['home_team', 'home_score', 'away_score', 'away_team', 'year', 'month', 'day']]
world_cup_matches = world_cup_matches[(world_cup_matches.year) >= 1930]
world_cup_matches = world_cup_matches[(world_cup_matches.year) <= 2018]
teams = ['Russia', 'Brazil', 'Australia', 'Iran', 'Japan', 'Saudi Arabia', 'South Korea', 'Egypt', 'Morocco', 'Nigeria', 'Senegal', 'Tunisia', 'Costa Rica', 'Mexico', 'Panama', 'Argentina', 'Colombia', 'Peru', 'Uruguay', 'Belgium', 'Croatia', 'Denmark', 'England', 'France', 'Germany', 'Iceland', 'Poland', 'Portugal', 'Serbia', 'Spain', 'Sweden', 'Switzerland']
teams.sort()
world_cup_matches = world_cup_matches[world_cup_matches.home_team.isin(teams) | world_cup_matches.away_team.isin(teams)]
world_cup_matches.head()

,home_team,home_score,away_score,away_team,year,month,day
1252,Spain,1,0,Czechoslovakia,1930,1,1
1253,Portugal,1,0,Czechoslovakia,1930,1,12
1256,Italy,4,2,Switzerland,1930,2,9
1259,Portugal,2,0,France,1930,2,23
1260,Germany,0,2,Italy,1930,3,2


In [118]:
world_cup_matches.describe()

,home_score,away_score,year,month,day
count,16594.000000,16594.000000,16594.000000,16594.000000,16594.000000
mean,1.688743,1.160118,1988.540918,6.590936,15.381885
std,1.656105,1.306737,21.683312,3.159086,8.537780
min,0.000000,0.000000,1930.000000,1.000000,1.000000
25%,1.000000,0.000000,1974.000000,4.000000,8.000000
50%,1.000000,1.000000,1993.000000,6.000000,15.000000
75%,2.000000,2.000000,2006.000000,9.000000,23.000000
max,31.000000,17.000000,2018.000000,12.000000,31.000000


In [123]:
data = {'Team': teams, 'GoalsScoredMean': [0 for i in range(32)], 'GoalsConcededMean': [0 for i in range(32)]}
stats = pd.DataFrame(data)
teams_att_power = []
teams_def_power = []
index = 0
for time in teams:
    team_goals = 0.0
    conceded_goals = 0.0
    team_matches = 0.0
    time_model = world_cup_matches[(world_cup_matches['home_team'] == time) | (world_cup_matches['away_team'] == time)]
    #print(time_model)
    for index, row in time_model.iterrows():
        if row['home_team'] == time:
            team_goals += row['home_score']
            conceded_goals += row['away_score']
        if row['away_team'] == time:
            team_goals += row['away_score']
            conceded_goals += row['home_score']
        team_matches += 1
    teams_att_power.append(team_goals/(team_matches))
    teams_def_power.append(conceded_goals/team_matches)
    index += 1
    stats.loc[stats['Team'] == time, 'GoalsScoredMean'] = team_goals/(team_matches)
    stats.loc[stats['Team'] == time, 'GoalsConcededMean'] = conceded_goals/(team_matches)
stats2 = stats.set_index('Team')
world_cup_matches['HomeAttack'] = [0.0 for i in range(len(world_cup_matches))]
world_cup_matches['HomeDefence'] = [0.0 for i in range(len(world_cup_matches))]
world_cup_matches['AwayAttack'] = [0.0 for i in range(len(world_cup_matches))]
world_cup_matches['AwayDefence'] = [0.0 for i in range(len(world_cup_matches))]

for index, row in world_cup_matches.iterrows():
    if (row['home_team'] in teams and row['away_team'] in teams):
        world_cup_matches.loc[index,'HomeAttack'] = stats2.loc[row['home_team'], 'GoalsScoredMean']
        world_cup_matches.loc[index,'HomeDefence'] = stats2.loc[row['home_team'], 'GoalsConcededMean']
        world_cup_matches.loc[index,'AwayAttack'] = stats2.loc[row['away_team'], 'GoalsScoredMean']
        world_cup_matches.loc[index,'AwayDefence'] = stats2.loc[row['away_team'], 'GoalsConcededMean']
    elif (row['home_team'] in teams and row['away_team'] not in teams):
        world_cup_matches.loc[index,'HomeAttack'] = stats2.loc[row['home_team'], 'GoalsScoredMean']
        world_cup_matches.loc[index,'HomeDefence'] = stats2.loc[row['home_team'], 'GoalsConcededMean']
    else:
        world_cup_matches.loc[index,'AwayAttack'] = stats2.loc[row['away_team'], 'GoalsScoredMean']
        world_cup_matches.loc[index,'AwayDefence'] = stats2.loc[row['away_team'], 'GoalsConcededMean']
world_cup_matches.head()

,home_team,home_score,away_score,away_team,year,month,day,HomeAttack,HomeDefence,AwayAttack,AwayDefence
1252,Spain,1,0,Czechoslovakia,1930,1,1,1.978852,0.913897,0.000000,0.000000
1253,Portugal,1,0,Czechoslovakia,1930,1,12,1.675170,1.137755,0.000000,0.000000
1256,Italy,4,2,Switzerland,1930,2,9,0.000000,0.000000,1.440510,1.586402
1259,Portugal,2,0,France,1930,2,23,1.675170,1.137755,1.779255,1.135638
1260,Germany,0,2,Italy,1930,3,2,2.238259,1.079038,0.000000,0.000000


In [ ]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

goal_model_data = pd.concat([world_cup_matches[['home_team','away_team', 'HomeAttack', 'HomeDefence', 'AwayAttack', 'AwayDefence', 'home_score']].rename(
            columns={'home_team':'team', 'away_team':'opponent', 'HomeAttack':'att', 'HomeDefence': 'defe', 'AwayAttack': 'away_att', 'AwayDefence': 'away_def','home_score':'goals'}),
           world_cup_matches[['away_team','home_team','HomeAttack', 'HomeDefence', 'AwayAttack', 'AwayDefence','away_score']].rename(
            columns={'away_team':'team', 'home_team':'opponent', 'HomeAttack':'att', 'HomeDefence': 'defe', 'AwayAttack': 'away_att', 'AwayDefence': 'away_def','away_score':'goals'})])
#print(goal_model_data)
poisson_model = smf.glm(formula="goals ~ team + opponent + att + defe + away_att + away_def", data=goal_model_data, 
                        family=sm.families.Poisson()).fit()
poisson_model.summary()

In [55]:
print(goal_model_data)

                         team             opponent  goals
1252                    Spain       Czechoslovakia      1
1253                 Portugal       Czechoslovakia      1
1256                    Italy          Switzerland      4
1259                 Portugal               France      2
1260                  Germany                Italy      0
1262                   France          Switzerland      3
1263                  England             Scotland      5
1265                   France              Belgium      1
1266              Switzerland              Hungary      2
1271              Netherlands              Belgium      2
1272              Switzerland              Germany      0
1274                  Germany              England      3
1275                  Belgium  Republic of Ireland      1
1276                   France       Czechoslovakia      2
1278                  Austria              England      0
1279                  Belgium          Netherlands      3
1280          

,home_team,home_score,away_score,away_team,year,month,day
1252,Spain,1,0,Czechoslovakia,1930,1,1
1253,Portugal,1,0,Czechoslovakia,1930,1,12
1256,Italy,4,2,Switzerland,1930,2,9
1259,Portugal,2,0,France,1930,2,23
1260,Germany,0,2,Italy,1930,3,2
